### EMA and CCI Strategy

In [8]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import CSV

In [36]:
data = pd.read_csv('AUDUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
11930,2022-04-29 16:45:00+01:00,0.710424,0.711288,0.710252,0.710308,0.710308,0
11931,2022-04-29 17:00:00+01:00,0.710252,0.710252,0.709084,0.709210,0.709210,0
11932,2022-04-29 17:15:00+01:00,0.709144,0.709824,0.709144,0.709310,0.709310,0
11933,2022-04-29 17:30:00+01:00,0.709215,0.709320,0.708843,0.709044,0.709044,0
11934,2022-04-29 17:45:00+01:00,0.709134,0.709471,0.708205,0.708386,0.708386,0


In [37]:
df = data.copy()

### Create strategy

In [13]:
ema_cci = ta.Strategy(
    name = 'EMA and CCI',
    description = 'EMA and CCI Strategy',
    ta = [
        {'kind': 'cci', 'length': 50, 'c': 0.015},
        {'kind': 'ema', 'length': 15},
        {'kind': 'ema', 'length': 25}
    ]
)
df.ta.strategy(ema_cci)
df

,timestamp,open,high,low,close,adj close,volumn,CCI_50_0.015,EMA_15,EMA_25
0,2021-11-04 17:00:00+00:00,1.349364,1.349364,1.348418,1.348963,1.348963,0,NaN,NaN,NaN
1,2021-11-04 17:15:00+00:00,1.349000,1.349400,1.348981,1.349273,1.349273,0,NaN,NaN,NaN
2,2021-11-04 17:30:00+00:00,1.349400,1.349437,1.348563,1.349109,1.349109,0,NaN,NaN,NaN
3,2021-11-04 17:45:00+00:00,1.348781,1.349273,1.348527,1.349273,1.349273,0,NaN,NaN,NaN
4,2021-11-04 18:00:00+00:00,1.349382,1.349546,1.349182,1.349546,1.349546,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11920,2022-04-29 16:45:00+01:00,1.256329,1.256929,1.256187,1.256581,1.256581,0,73.552861,1.255611,1.255292
11921,2022-04-29 17:00:00+01:00,1.256329,1.256644,1.255824,1.255950,1.255950,0,59.118419,1.255654,1.255343
11922,2022-04-29 17:15:00+01:00,1.256281,1.257719,1.256281,1.256787,1.256787,0,84.364797,1.255795,1.255454
11923,2022-04-29 17:30:00+01:00,1.256266,1.257150,1.256266,1.256913,1.256913,0,78.786998,1.255935,1.255566


In [7]:
df[df['CCI_50_0.015'] < -100]

,timestamp,open,high,low,close,adj close,volumn,CCI_50_0.015,EMA_15,EMA_25
60,2021-11-05 08:00:00+00:00,1.350311,1.350311,1.347346,1.347836,1.347836,0,-160.964758,1.349626,1.349604
61,2021-11-05 08:15:00+00:00,1.347001,1.347618,1.346167,1.346674,1.346674,0,-350.807523,1.349257,1.349378
62,2021-11-05 08:30:00+00:00,1.346801,1.347309,1.345641,1.345641,1.345641,0,-378.421680,1.348805,1.349091
63,2021-11-05 08:45:00+00:00,1.345316,1.346493,1.345062,1.345062,1.345062,0,-393.989439,1.348337,1.348781
64,2021-11-05 09:00:00+00:00,1.345551,1.345551,1.344050,1.344393,1.344393,0,-401.465306,1.347844,1.348444
...,...,...,...,...,...,...,...,...,...,...
11816,2022-04-28 14:45:00+01:00,1.243286,1.244153,1.242900,1.243626,1.243626,0,-145.161877,1.245463,1.247125
11817,2022-04-28 15:00:00+01:00,1.243394,1.243425,1.242005,1.242082,1.242082,0,-153.967180,1.245040,1.246737
11818,2022-04-28 15:15:00+01:00,1.242190,1.243565,1.241989,1.243565,1.243565,0,-133.963756,1.244856,1.246493
11819,2022-04-28 15:30:00+01:00,1.243147,1.243147,1.241511,1.242390,1.242390,0,-137.017768,1.244547,1.246178


### Create Signal

In [38]:
class BacktestEMAandCCI:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        self.atr = params['atr']
        
        self.current_action = 'close'
        self.ema_1 = params['ema_1']
        self.ema_2 = params['ema_2']
        self.cci_len = params['cci_len']
        self.cci_c = params['cci_c']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        self.plot = params['plot']
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
        
    def plot_order(self, order):
        pass
        
    def backtest(self):
        n_bars = 3
        ema_cci = ta.Strategy(
            name = 'EMA and CCI',
            description = 'EMA and CCI Strategy',
            ta = [
                {'kind': 'cci', 'length': self.cci_len, 'c': self.cci_c},
                {'kind': 'atr', 'length': self.atr},
                {'kind': 'ema', 'length': self.ema_1},
                {'kind': 'ema', 'length': self.ema_2}
            ]
        )
        
        self.df.ta.strategy(ema_cci)
        self.df = self.df.copy().dropna()
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['action'] = None
        
        self.df['entries'] = False
        self.df['exits'] = False
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['close']
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
            
                if row['EMA_'+str(self.ema_1)] > row['EMA_'+str(self.ema_2)]:
                    # check if previous n bars ema_1 < ema_2 then buy.
                    if index > n_bars:
                        for pre_index, pre_row in self.df.iloc[index-n_bars:index].iterrows():
                            if (pre_row['EMA_'+str(self.ema_1)] < pre_row['EMA_'+str(self.ema_2)]) and \
                            (pre_row['CCI_'+str(self.cci_len)+'_'+str(self.cci_c)] > 100):
                                self.df.at[index, 'action'] = 'buy'
                                self.df.at[index, 'entries'] = True
                                self.df.at[index, 'price_action'] = row['open']
                                self.order['action'] = 'buy'
                                
                                self.order['open time'] = row['timestamp']
                                self.order['open'] = row['open']
                                self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
                                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                                self.current_action = 'buy'
                        
                elif row['EMA_'+str(self.ema_1)] < row['EMA_'+str(self.ema_2)]:
                    # check if previous n bars ema_1 > ema_2 then sell.
                    if index > n_bars:
                        for pre_index, pre_row in self.df.iloc[index-n_bars:index].iterrows():
                            if (pre_row['EMA_'+str(self.ema_1)] > pre_row['EMA_'+str(self.ema_2)]) and \
                            (pre_row['CCI_'+str(self.cci_len)+'_'+str(self.cci_c)] < -100):
                                self.df.at[index, 'action'] = 'sell'
                                self.df.at[index, 'short_entries'] = True
                                self.df.at[index, 'price_action'] = row['open']
                                self.order['action'] = 'sell'
                                
                                self.order['open time'] = row['timestamp']
                                self.order['open'] = row['open']
                                self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
                                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                                self.current_action = 'sell'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema_1: {self.ema_1}, ema_2: {self.ema_2}, cci: {self.cci_len} {self.cci_c}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [39]:
pair_test = BacktestEMAandCCI({
    'cci_len': 50,
    'cci_c': 0.015,
    'ema_1': 15,
    'ema_2': 25,
    'atr': 14,
    'pip_value': 0.001,
    'rr': 2,
    'plot': False,
    'df': df.copy()
})
pair_test.backtest()

{'win_rate': 14.29,
 'info': 'total orders: 7\nema_1: 15, ema_2: 25, cci: 50 0.015, gain: -4\n',
 'total_order': 7,
 'rr': 2,
 'gain': -4,
 'unknow': []}